# CSYE7245_DGA_Detection Assignment4

## Part C.B - MLLIB AND MACHINE LEARNING


### Course Information
* **School**: College of Engineering, Northeastern University
* **Course Name**: Big-Data Systems and Intelligence Analytics
* **Professor**: Nik Brown

### Group Members
- Haimin Zhang
- Lixi Zhou
- Shiqi Dai

### Requirements
Apply a MLlib and Machine Learning analysis of your project data.

In [2]:
import gc
import sys
import math
import collections
from tld import get_tld
import re
from publicsuffixlist import PublicSuffixList
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import IndexToString
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [3]:
# read csv and display csv
mixed_domains = sqlContext.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/mixed_domain.csv')
display(mixed_domains)
pandas_mixed_domains = mixed_domains.toPandas()
del mixed_domains
gc.collect()
# mixed_domain.printSchema()
# mixed_domain.describe().show()

DGA_family,Domain,Type
banjori,vrxhererwyatanb.com,DGA
banjori,rjdwmachuslazaroqok.com,DGA
none,mte.gov.br,Normal
none,animefangirl00.tumblr.com,Normal
none,seacenter.ir,Normal
Post,1xhkzo0vu7c96fwf07o1o9wjau.org,DGA
none,d-a-z.hr,Normal
none,asternic.net,Normal
rovnix,f8qlliz2qyitk5hmpl.biz,DGA
emotet,snpwmbhvvsnjoijr.eu,DGA


In [4]:

psl = PublicSuffixList()

# Load Valid Top Level Domains data
def load_topLevelDomain():
  topLevelDomain = []
  with open('/dbfs/FileStore/tables/tlds_alpha_by_domain-402ba.txt', 'r') as content:
      for line in content:
          topLevelDomain.append((line.strip('\n')))
  return topLevelDomain


def ignoreVPS(domain):
    # Return the rest of domain after ignoring the Valid Public Suffixes:
    validPublicSuffix = '.' + psl.publicsuffix(domain)
    if len(validPublicSuffix) < len(domain):
         # If it has VPS
        subString = domain[0: domain.index(validPublicSuffix)]  
    elif len(validPublicSuffix) == len(domain):
        return 0
    else:
        # If not
        subString = domain
    
    return subString

def typeTo_Binary(type):
  # Convert Type to Binary variable DGA = 1, Normal = 0
  if type == 'DGA':
    return 1
  else:
    return 0

def domain_length(domain):
  # Generate Domain Name Length (DNL)
  return len(domain)

def subdomains_number(domain):
  # Generate Number of Subdomains (NoS)
  subdomain = ignoreVPS(domain)
  return (subdomain.count('.') + 1)

def subdomain_length_mean(domain):
  # enerate Subdomain Length Mean (SLM) 
  subdomain = ignoreVPS(domain)
  result = (len(subdomain) - subdomain.count('.')) / (subdomain.count('.') + 1)
  return result

def has_www_prefix(domain):
  # Generate Has www Prefix (HwP)
  if domain.split('.')[0] == 'www':
    return 1
  else:
    return 0
  
def has_hvltd(domain):
  topLevelDomain = load_topLevelDomain()
  # Generate Has a Valid Top Level Domain (HVTLD)
  if domain.split('.')[len(domain.split('.')) - 1].upper() in topLevelDomain:
    return 1
  else:
    return 0
  
def contains_single_character_subdomain(domain):
  # Generate Contains Single-Character Subdomain (CSCS) 
  domain = ignoreVPS(domain)
  str_split = domain.split('.')
  minLength = len(str_split[0])
  for i in range(0, len(str_split) - 1):
      minLength = len(str_split[i]) if len(str_split[i]) < minLength else minLength
  if minLength == 1:
    return 1
  else:
    return 0

def contains_TLD_subdomain(domain):
  # Generate Contains TLD as Subdomain (CTS)
  subdomain = ignoreVPS(domain)
  str_split = subdomain.split('.')
  topLevelDomain = load_topLevelDomain()
  for i in range(0, len(str_split) - 1):
        if str_split[i].upper() in topLevelDomain:
            return 1
  return 0

def underscore_ratio(domain):
  # Generate Underscore Ratio (UR) on dataset
  subString = ignoreVPS(domain)
  result = subString.count('_') / (len(subString) - subString.count('.'))
  return result

def contains_IP_address(domain):
  # Generate Contains IP Address (CIPA) on datasetx
    splitSet = domain.split('.')
    for element in splitSet:
        if(re.match("\d+", element)) == None:
            return 0
    return 1  

def contains_digit(domain):
  """
   Contains Digits 
  """
  subdomain = ignoreVPS(domain)
  for item in subdomain:
    if item.isdigit():
      return 1
  return 0

def vowel_ratio(domain):
  """
  calculate Vowel Ratio 
  """
  VOWELS = set('aeiou')
  v_counter = 0
  a_counter = 0
  subdomain = ignoreVPS(domain)
  for item in subdomain:
    if item.isalpha():
      a_counter+=1
      if item in VOWELS:
        v_counter+=1
  if a_counter>1:
    ratio = v_counter/a_counter
    return ratio

def digit_ratio(domain):
  """
  calculate digit ratio
  """
  d_counter = 0
  counter = 0
  subdomain = ignoreVPS(domain)
  for item in subdomain:
    if item.isalpha() or item.isdigit():
      counter+=1
      if item.isdigit():
        d_counter+=1
  if counter>1:
    ratio = d_counter/counter
    return ratio
  
def prc_rrc(domain):
  """
  calculate the Ratio of Repeated Characters in a subdomain
  """
  subdomain = ignoreVPS(domain)
#   subdomain =''.join(re.findall('[a-zA-Z]+', subdomain)) 
  subdomain = re.sub("[.]", "", subdomain)
  char_num=0
  repeated_char_num=0
  d = collections.defaultdict(int)
  for c in list(subdomain):
      d[c] += 1
  for item in d:
    char_num +=1
    if d[item]>1:
      repeated_char_num +=1
  ratio = repeated_char_num/char_num
  return ratio

def prc_rcc(domain):
  """
  calculate the Ratio of Consecutive Consonants
  """
  VOWELS = set('aeiou')
  counter = 0
  cons_counter=0
  subdomain = ignoreVPS(domain)
#   subdomain =''.join(re.findall('[a-zA-Z]+', subdomain)) 
  for item in subdomain:
    i = 0
    if item.isalpha() and item not in VOWELS:
      counter+=1
    else:
      if counter>1:
        cons_counter+=counter
      counter=0
    i+=1
  if i==len(subdomain) and counter>1:
    cons_counter+=counter
  ratio = cons_counter/len(subdomain)
  return ratio

def prc_rcd(domain):
  """
  calculate the ratio of consecutive digits
  """
  counter = 0
  digit_counter=0
  subdomain = ignoreVPS(domain)
#   subdomain =''.join(re.findall('[a-zA-Z]+', subdomain)) 
  for item in subdomain:
    i = 0
    if item.isdigit():
      counter+=1
    else:
      if counter>1:
        digit_counter+=counter
      counter=0
    i+=1
  if i==len(subdomain) and counter>1:
    digit_counter+=counter
  ratio = digit_counter/len(subdomain)
  return ratio

def prc_entropy(domain):
    """
    calculate the entropy of subdomain
    :param domain_str: subdomain
    :return: the value of entropy
    """
    subdomain = ignoreVPS(domain)
    # get probability of chars in string
    prob = [float(subdomain.count(c)) / len(subdomain) for c in dict.fromkeys(list(subdomain))]

    # calculate the entropy
    entropy = - sum([p * math.log(p) / math.log(2.0) for p in prob])
    return entropy

In [5]:
def extract_features():
  pandas_mixed_domains['DNL'] = pandas_mixed_domains['Domain'].apply(lambda x: domain_length(x))
  pandas_mixed_domains['NoS'] = pandas_mixed_domains['Domain'].apply(lambda x: subdomains_number(x))
  pandas_mixed_domains['SLM'] = pandas_mixed_domains['Domain'].apply(lambda x: subdomain_length_mean(x))
  pandas_mixed_domains['HwP'] = pandas_mixed_domains['Domain'].apply(lambda x: has_www_prefix(x))
  pandas_mixed_domains['HVTLD'] = pandas_mixed_domains['Domain'].apply(lambda x: has_hvltd(x))
  pandas_mixed_domains['CSCS'] = pandas_mixed_domains['Domain'].apply(lambda x: contains_single_character_subdomain(x))
  pandas_mixed_domains['CTS'] = pandas_mixed_domains['Domain'].apply(lambda x: contains_TLD_subdomain(x))
  pandas_mixed_domains['UR'] = pandas_mixed_domains['Domain'].apply(lambda x: underscore_ratio(x))
  pandas_mixed_domains['CIPA'] = pandas_mixed_domains['Domain'].apply(lambda x: contains_IP_address(x))
  pandas_mixed_domains['contains_digit']= pandas_mixed_domains['Domain'].apply(lambda x:contains_digit(x))
  pandas_mixed_domains['vowel_ratio']= pandas_mixed_domains['Domain'].apply(lambda x:vowel_ratio(x))
  pandas_mixed_domains['digit_ratio']= pandas_mixed_domains['Domain'].apply(lambda x:digit_ratio(x))
  pandas_mixed_domains['RRC']= pandas_mixed_domains['Domain'].apply(lambda x:prc_rrc(x))
  pandas_mixed_domains['RCC']= pandas_mixed_domains['Domain'].apply(lambda x:prc_rcc(x))
  pandas_mixed_domains['RCD']= pandas_mixed_domains['Domain'].apply(lambda x:prc_rcd(x))
  pandas_mixed_domains['Entropy']= pandas_mixed_domains['Domain'].apply(lambda x:prc_entropy(x))
#   print(pandas_mixed_domains)

In [6]:
extract_features()
mixed_domains = spark.createDataFrame(pandas_mixed_domains)
mixed_domains =mixed_domains.drop('DGA_family')
del pandas_mixed_domains
gc.collect()
# mixed_domains.show()

Out[ 7 ]: 150

In [7]:
# This part is to save  featured dataframe to csv file. When restarts again, users would not wait to the creation of featured data frame
# mixed_domains.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("/FileStore/tables/featured_domains6.csv")
# mixed_domains = sqlContext.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/featured_domains6.csv')
# mixed_domains.show()
# del mixed_domains2
# gc.collect()

+--------------------+------+---+---+----+---+-----+----+---+---+----+--------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+
 Domain| Type|DNL|NoS| SLM|HwP|HVTLD|CSCS|CTS| UR|CIPA|contains_digit| vowel_ratio| digit_ratio| RRC| RCC| RCD| Entropy|
+--------------------+------+---+---+----+---+-----+----+---+---+----+--------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+
 vrxhererwyatanb.com| DGA| 19| 1|15.0| 0| 1| 0| 0|0.0| 0| 0|0.26666666666666666| 0.0| 0.2727272727272727| 0.4666666666666667| 0.0| 3.32323142879762|
rjdwmachuslazaroq...| DGA| 23| 1|19.0| 0| 1| 0| 0|0.0| 0| 0| 0.3157894736842105| 0.0| 0.2| 0.47368421052631576| 0.0|3.7871439606981414|
 mte.gov.br|Normal| 10| 1| 3.0| 0| 1| 0| 0|0.0| 0| 0| 0.3333333333333333| 0.0| 0.0| 0.6666666666666666| 0.0| 1.584962500721156|
animefangirl00.tu...|Normal| 25| 2|10.0| 0| 1| 0| 0|0.0| 0| 1| 0.3333333333333333| 0.1| 0.5384615384615384| 0.19047619047619047|0.09523809523809523|3.7256507561120937|
 seacenter.ir|Normal| 12| 1| 9.0| 0| 1| 0| 0|0.0| 0| 0| 0.4444444444444444| 0.0|0.14285714285714285| 0.2222222222222222| 0.0|2.6416041678685938|
1xhkzo0vu7c96fwf0...| DGA| 30| 1|26.0| 0| 1| 0| 0|0.0| 0| 1|0.35294117647058826|0.34615384615384615|0.47058823529411764| 0.34615384615384615|0.15384615384615385| 3.979097891134805|
 d-a-z.hr|Normal| 8| 1| 5.0| 0| 1| 0| 0|0.0| 0| 0| 0.3333333333333333| 0.0| 0.25| 0.0| 0.0|1.9219280948873625|
 asternic.net|Normal| 12| 1| 8.0| 0| 1| 0| 0|0.0| 0| 0| 0.375| 0.0| 0.0| 0.5| 0.0| 3.0|
f8qlliz2qyitk5hmp...| DGA| 22| 1|18.0| 0| 1| 0| 0|0.0| 0| 1|0.13333333333333333|0.16666666666666666|0.21428571428571427| 0.3888888888888889| 0.0| 3.683542362433231|
 snpwmbhvvsnjoijr.eu| DGA| 19| 1|16.0| 0| 1| 0| 0|0.0| 0| 0| 0.125| 0.0| 0.3333333333333333| 0.75| 0.0| 3.5|
 cdmstudy.site|Normal| 13| 1| 8.0| 0| 1| 0| 0|0.0| 0| 0| 0.125| 0.0|0.14285714285714285| 0.625| 0.0| 2.75|
tjjlsikathrinezad...| DGA| 21| 1|17.0| 0| 1| 0| 0|0.0| 0| 0|0.29411764705882354| 0.0| 0.3076923076923077| 0.47058823529411764| 0.0|3.6168746059562227|
 alfaman.org|Normal| 11| 1| 7.0| 0| 1| 0| 0|0.0| 0| 0|0.42857142857142855| 0.0| 0.2| 0.2857142857142857| 0.0|2.1280852788913944|
d6y5v2gzha3tnx1rz...| DGA| 22| 1|18.0| 0| 1| 0| 0|0.0| 0| 1|0.07692307692307693| 0.2777777777777778| 0.125| 0.3333333333333333| 0.0|3.9477027792200903|
1okgbh8tpc1cm61r1...| DGA| 22| 1|18.0| 0| 1| 0| 0|0.0| 0| 1|0.09090909090909091| 0.3888888888888889|0.14285714285714285| 0.5| 0.1111111111111111|3.6143694458867572|
d37b7a931cee83903...| DGA| 37| 1|34.0| 0| 1| 0| 0|0.0| 0| 1| 0.3076923076923077| 0.6176470588235294| 0.8333333333333334|0.058823529411764705| 0.5294117647058824| 3.408239784491492|
astrologiaarqueti...|Normal| 35| 2|15.0| 0| 1| 0| 0|0.0| 0| 0|0.43333333333333335| 0.0| 0.5333333333333333| 0.2903225806451613| 0.0|3.7843684875968617|
ednyilmgbrbcbocqq...| DGA| 22| 1|18.0| 0| 1| 0| 0|0.0| 0| 0|0.16666666666666666| 0.0| 0.3076923076923077| 0.6111111111111112| 0.0| 3.57243125132212|
 ooljpartbulkyf.com| DGA| 18| 1|14.0| 0| 1| 0| 0|0.0| 0| 0| 0.2857142857142857| 0.0|0.16666666666666666| 0.42857142857142855| 0.0| 3.521640636343319|
 dhruphost.com|Normal| 13| 1| 9.0| 0| 1| 0| 0|0.0| 0| 0| 0.2222222222222222| 0.0| 0.125| 0.5555555555555556| 0.0|2.9477027792200903|
+--------------------+------+---+---+----+---+-----+----+---+---+----+--------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+
only showing top 20 rows

In [8]:
def evaluate_models(predictions):
      # Select example rows to display.
      predictions.select("Domain","Type","features","predictedLabel").show(50)

      # Select (prediction, true label) and compute test error
      evaluator = MulticlassClassificationEvaluator(
          labelCol="indexed_type", predictionCol="prediction", metricName="accuracy")
      accuracy = evaluator.evaluate(predictions)
      print("Test Error = %g" % (1.0 - accuracy))

In [9]:
type_indexer = StringIndexer(inputCol="Type", outputCol="indexed_type").fit(mixed_domains)
indexed_type_domains = type_indexer.transform(mixed_domains)
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = indexed_type_domains.randomSplit([0.7, 0.3])

# del mixed_domains
# gc.collect()

assembler = VectorAssembler(
    inputCols=["DNL","NoS","SLM","HwP","HVTLD","CSCS","CTS","UR","CIPA","Entropy", "RRC", "RCC","RCD","contains_digit","vowel_ratio","digit_ratio"],
    outputCol="features", handleInvalid = "skip")

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",labels=type_indexer.labels)


# Build Random forest classifier
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexed_type", featuresCol="features", numTrees=10)

# # # Chain indexers and forest in a Pipeline
rf_pipeline = Pipeline(stages=[assembler,rf, labelConverter])

# Use CV to choose the best hyper parameters
rf_paramGrid = ParamGridBuilder().addGrid(rf.maxDepth,[10, 15,20]).addGrid(rf.numTrees,[20,30]).build()

rf_evaluator = MulticlassClassificationEvaluator(labelCol="indexed_type", predictionCol="prediction", metricName="accuracy")

rf_crossval = CrossValidator(estimator=rf_pipeline,estimatorParamMaps=rf_paramGrid,evaluator=rf_evaluator,numFolds=3)  
# use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
rf_cvModel = rf_crossval.fit(trainingData)

# Make prediction with cv
rf_predictions = rf_cvModel.transform(testData)

# # # Train model.  This also runs the indexers.
# rf_model = rf_pipeline.fit(trainingData)

# # # Make predictions.
# rf_predictions = rf_model.transform(testData)
evaluate_models(rf_predictions)


+--------------------+------+--------------------+--------------+
 Domain| Type| features|predictedLabel|
+--------------------+------+--------------------+--------------+
 000domains.com|Normal|[14.0,1.0,10.0,0....| Normal|
 000webhost.com|Normal|[14.0,1.0,10.0,0....| Normal|
 0086cctv.com|Normal|(16,[0,1,2,4,9,10...| Normal|
014d9e57888d4e2b7...| DGA|[36.0,1.0,32.0,0....| DGA|
 01ny.cn|Normal|(16,[0,1,2,4,9,12...| Normal|
 024yeya.com|Normal|[11.0,1.0,7.0,0.0...| Normal|
 02dual.com|Normal|(16,[0,1,2,4,9,12...| Normal|
 0575ls.cn|Normal|(16,[0,1,2,4,9,10...| Normal|
 05a3cl2jstin.org| DGA|[16.0,1.0,12.0,0....| DGA|
 08eins.swiss|Normal|(16,[0,1,2,4,9,12...| Normal|
 09xy.sk|Normal|(16,[0,1,2,4,9,12...| Normal|
 0bin.net|Normal|(16,[0,1,2,4,9,13...| Normal|
 0catch.com|Normal|(16,[0,1,2,4,9,10...| Normal|
 0e4e42se0i4m.net| DGA|[16.0,1.0,12.0,0....| DGA|
 0hqvs9qr0x2z.top| DGA|(16,[0,1,2,4,9,10...| DGA|
 0puzw1anslen.org| DGA|[16.0,1.0,12.0,0....| DGA|
 0qg20mc606se.org| DGA|[16.0,1.0,12.0,0....| DGA|
 0teefetch.us|Normal|(16,[0,1,2,4,9,10...| Normal|
 0vvvv0.tumblr.com|Normal|[17.0,2.0,6.0,0.0...| Normal|
 0zz0.com|Normal|(16,[0,1,2,4,9,10...| Normal|
 1-ofd.ru|Normal|(16,[0,1,2,4,9,13...| Normal|
 1000more.net|Normal|[12.0,1.0,8.0,0.0...| Normal|
1001-nuits-enchan...|Normal|[24.0,1.0,21.0,0....| Normal|
 1001fonts.com|Normal|[13.0,1.0,9.0,0.0...| Normal|
 1001hry.cz|Normal|(16,[0,1,2,4,9,10...| Normal|
 1001jogos.pt|Normal|[12.0,1.0,9.0,0.0...| Normal|
 1001modelkits.com|Normal|[17.0,1.0,13.0,0....| Normal|
 1001receitas.com|Normal|[16.0,1.0,12.0,0....| Normal|
 1001sprueche.com|Normal|[16.0,1.0,12.0,0....| DGA|
 100appsnow.com|Normal|[14.0,1.0,10.0,0....| Normal|
100escaperswalkth...|Normal|[26.0,1.0,22.0,0....| Normal|
 100freemb.com|Normal|[13.0,1.0,9.0,0.0...| Normal|
 100gbtube.com|Normal|[13.0,1.0,9.0,0.0...| Normal|
 100how.com|Normal|[10.0,1.0,6.0,0.0...| Normal|
 100jang.com|Normal|[11.0,1.0,7.0,0.0...| Normal|
 100kmwalker-etc.com|Normal|[19.0,1.0,15.0,0....| DGA|
 100obmrzlireki.si|Normal|[17.0,1.0,14.0,0....| Normal|
 100pepinos.com.br|Normal|[17.0,1.0,10.0,0....| Normal|
 100podelok.com|Normal|[14.0,1.0,10.0,0....| Normal|
 100pro.su|Normal|[9.0,1.0,6.0,0.0,...| Normal|
100searchengines.com|Normal|[20.0,1.0,16.0,0....| Normal|
 100solucoessika.pt|Normal|[18.0,1.0,15.0,0....| Normal|
 101dogovor.ru|Normal|[13.0,1.0,10.0,0....| Normal|
 1024pix.github.io|Normal|(16,[0,1,2,4,9,12...| Normal|
103o4gwzlefapj581...| DGA|[29.0,1.0,25.0,0....| DGA|
 1040now.net|Normal|[11.0,1.0,7.0,0.0...| Normal|
1049520dvgg5mqd9u...| DGA|[29.0,1.0,25.0,0....| DGA|
104ck1413ngrxj18j...| DGA|[32.0,1.0,28.0,0....| DGA|
104wce8gp4csn14mo...| DGA|[31.0,1.0,27.0,0....| DGA|
1058djigid6i1cykl...| DGA|[30.0,1.0,26.0,0....| DGA|
+--------------------+------+--------------------+--------------+
only showing top 50 rows

Test Error = 0.0948686

In [10]:
# Gradient-boosted tree classifier

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexed_type", featuresCol="features", maxIter=10)

# Chain indexers and GBT in a Pipeline
gbt_pipeline = Pipeline(stages=[assembler,gbt, labelConverter])

# Use CV to choose the best hyper parameters
gbt_paramGrid = ParamGridBuilder().addGrid(gbt.maxDepth,[15, 20]).addGrid(gbt.maxIter,[20,30]).build()

gbt_evaluator = MulticlassClassificationEvaluator(labelCol="indexed_type", predictionCol="prediction", metricName="accuracy")

gbt_crossval = CrossValidator(estimator=gbt_pipeline,estimatorParamMaps=gbt_paramGrid,evaluator=gbt_evaluator,numFolds=3)  
# use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
gbt_cvModel = gbt_crossval.fit(trainingData)

# Train model.  This also runs the indexers.
# gbt_model = gbt_pipeline.fit(trainingData)

# Make predictions.
# gbt_predictions = gbt_model.transform(testData)
gbt_predictions = gbt_cvModel.transform(testData)
evaluate_models(gbt_predictions)

In [11]:
evaluate_models(gbt_predictions)

+--------------------+------+--------------------+--------------+
 Domain| Type| features|predictedLabel|
+--------------------+------+--------------------+--------------+
 000webhost.com|Normal|[14.0,1.0,10.0,0....| Normal|
 02edu.ru|Normal|(16,[0,1,2,4,9,12...| Normal|
 050webhosting.com|Normal|[17.0,1.0,13.0,0....| Normal|
 05a3cl2jstin.org| DGA|[16.0,1.0,12.0,0....| DGA|
 0716edu.net|Normal|(16,[0,1,2,4,9,12...| Normal|
 0769pf.com|Normal|(16,[0,1,2,4,9,12...| Normal|
 0bi8.com|Normal|(16,[0,1,2,4,9,13...| Normal|
0c3ayrqqy5fbc1c2.biz| DGA|[20.0,1.0,16.0,0....| DGA|
 0e4e42se0i4m.net| DGA|[16.0,1.0,12.0,0....| DGA|
 0hqvs9qr0x2z.top| DGA|(16,[0,1,2,4,9,10...| DGA|
 0teefetch.us|Normal|(16,[0,1,2,4,9,10...| Normal|
 1-1ads.com|Normal|(16,[0,1,2,4,9,10...| Normal|
 1-language.com|Normal|[14.0,1.0,10.0,0....| Normal|
 1000foruns.com|Normal|[14.0,1.0,10.0,0....| Normal|
 1000lnw.org|Normal|(16,[0,1,2,4,9,10...| Normal|
 1000note.it|Normal|[11.0,1.0,8.0,0.0...| Normal|
 1000projects.com|Normal|[16.0,1.0,12.0,0....| Normal|
1001-nuits-enchan...|Normal|[24.0,1.0,21.0,0....| Normal|
 1001fonts.com|Normal|[13.0,1.0,9.0,0.0...| Normal|
 1001student.ru|Normal|[14.0,1.0,11.0,0....| Normal|
 1004ks.com|Normal|(16,[0,1,2,4,9,10...| Normal|
 100daychallenge.com|Normal|[19.0,1.0,15.0,0....| Normal|
100jaarorthopedag...|Normal|[25.0,1.0,22.0,0....| Normal|
 100kmwalker-etc.com|Normal|[19.0,1.0,15.0,0....| DGA|
100meninasnegras.com|Normal|[20.0,1.0,16.0,0....| Normal|
 100negocios.com|Normal|[15.0,1.0,11.0,0....| Normal|
 100obmrzlireki.si|Normal|[17.0,1.0,14.0,0....| Normal|
100percentdesign....|Normal|[22.0,1.0,16.0,0....| Normal|
 100soft.us|Normal|[10.0,1.0,7.0,0.0...| Normal|
 101dogovor.ru|Normal|[13.0,1.0,10.0,0....| Normal|
 101jurist.com|Normal|[13.0,1.0,9.0,0.0...| Normal|
 101kofemashina.ru|Normal|[17.0,1.0,14.0,0....| Normal|
 101million.com|Normal|[14.0,1.0,10.0,0....| Normal|
 101mobility.com|Normal|[15.0,1.0,11.0,0....| Normal|
 101xp.com|Normal|(16,[0,1,2,4,9,10...| Normal|
 1031maxfm.com|Normal|[13.0,1.0,9.0,0.0...| Normal|
103hwk31izj5cx1v7...| DGA|[32.0,1.0,28.0,0....| DGA|
103o4gwzlefapj581...| DGA|[29.0,1.0,25.0,0....| DGA|
103v9z8147m67g1tk...| DGA|[32.0,1.0,28.0,0....| DGA|
104cloud-my.share...|Normal|[26.0,2.0,10.5,0....| Normal|
104dg2i2oc14tlkjj...| DGA|[29.0,1.0,25.0,0....| DGA|
105gf52vokom51hw2...| DGA|[31.0,1.0,27.0,0....| DGA|
105oifx8p2skc1av7...| DGA|[31.0,1.0,27.0,0....| DGA|
105qj3g6a39xprv8f...| DGA|[28.0,1.0,24.0,0....| DGA|
106tdbt1m9q4az1dd...| DGA|[31.0,1.0,27.0,0....| DGA|
106te94851os91qvw...| DGA|[31.0,1.0,27.0,0....| DGA|
107dzvs6vwc6bkmkp...| DGA|[30.0,1.0,26.0,0....| DGA|
107ugtl1c4js4n1wr...| DGA|[32.0,1.0,28.0,0....| DGA|
1080ptorrents.blo...|Normal|[26.0,1.0,13.0,0....| Normal|
 108matcha-saro.com|Normal|[18.0,1.0,14.0,0....| Normal|
+--------------------+------+--------------------+--------------+
only showing top 50 rows

Test Error = 0.084618

In [12]:
# Naive Bayes Classifer
nb = NaiveBayes(featuresCol='features', labelCol='indexed_type',smoothing=1.0, modelType="multinomial")

# Chain indexers and forest in a Pipeline
nb_pipeline = Pipeline(stages=[assembler,nb, labelConverter])

# Use CV to choose the best hyper parameters
nb_paramGrid = ParamGridBuilder().addGrid(nb.smoothing,[0.1,0.3,0.5]).build()

nb_evaluator = MulticlassClassificationEvaluator(labelCol="indexed_type", predictionCol="prediction", metricName="accuracy")

nb_crossval = CrossValidator(estimator=nb_pipeline,estimatorParamMaps=nb_paramGrid,evaluator=nb_evaluator,numFolds=3)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
nb_cvModel = nb_crossval.fit(trainingData)

# Train model.  This also runs the indexers.
# model = nb_pipeline.fit(trainingData)

# Make predictions.
# nb_predictions = model.transform(testData)
nb_predictions = nb_cvModel.transform(testData)

evaluate_models(nb_predictions)

+--------------------+------+--------------------+--------------+
 Domain| Type| features|predictedLabel|
+--------------------+------+--------------------+--------------+
006a039c957c142bb...|Normal|[21.0,1.0,17.0,0....| DGA|
014d9e57888d4e2b7...| DGA|[37.0,1.0,32.0,0....| DGA|
 021-nekretnine.rs|Normal|[17.0,1.0,14.0,0....| DGA|
 05uvcxe74t6n.net| DGA|[16.0,1.0,12.0,0....| DGA|
 0769pf.com|Normal|(16,[0,1,2,4,9,12...| DGA|
 08eins.swiss|Normal|(16,[0,1,2,4,9,12...| DGA|
 09qnstersdq3.org| DGA|[16.0,1.0,12.0,0....| DGA|
0c3ayrqqy5fbc1c2.biz| DGA|[20.0,1.0,16.0,0....| DGA|
 0dayku.cn|Normal|(16,[0,1,2,4,9,11...| DGA|
 0hqvs9qr0x2z.top| DGA|(16,[0,1,2,4,9,10...| DGA|
 0p2fshizktm7.top| DGA|(16,[0,1,2,4,9,11...| DGA|
 0plati.today|Normal|(16,[0,1,2,4,9,11...| DGA|
 0teefetch.us|Normal|(16,[0,1,2,4,9,10...| DGA|
 0zz0.com|Normal|(16,[0,1,2,4,9,10...| DGA|
 1-1ads.com|Normal|(16,[0,1,2,4,9,10...| DGA|
 1-ofd.ru|Normal|(16,[0,1,2,4,9,13...| DGA|
 1000-annonces.com|Normal|[17.0,1.0,13.0,0....| DGA|
 1000-slow.pl|Normal|[12.0,1.0,9.0,0.0...| DGA|
 1000corks.com|Normal|[13.0,1.0,9.0,0.0...| DGA|
 1000lnw.org|Normal|(16,[0,1,2,4,9,10...| DGA|
 1000more.net|Normal|[12.0,1.0,8.0,0.0...| DGA|
 1001receitas.com|Normal|[16.0,1.0,12.0,0....| DGA|
1001spankingfanta...|Normal|[35.0,2.0,15.0,0....| DGA|
 100benzopil.ru|Normal|[14.0,1.0,11.0,0....| DGA|
100ehca1gfoq2z1ny...| DGA|[31.0,1.0,27.0,0....| DGA|
 100how.com|Normal|[10.0,1.0,6.0,0.0...| DGA|
 100megabit.ru|Normal|[13.0,1.0,10.0,0....| DGA|
 100mountain.com|Normal|[15.0,1.0,11.0,0....| DGA|
 100podelok.com|Normal|[14.0,1.0,10.0,0....| DGA|
 100yl.net|Normal|(16,[0,1,2,4,9,10...| DGA|
 101million.com|Normal|[14.0,1.0,10.0,0....| DGA|
 101qs.com|Normal|(16,[0,1,2,4,9,10...| DGA|
 101sz1.pw|Normal|[9.0,1.0,6.0,0.0,...| DGA|
 1033theeagle.com|Normal|[16.0,1.0,12.0,0....| DGA|
103hwk31izj5cx1v7...| DGA|[32.0,1.0,28.0,0....| DGA|
1049520dvgg5mqd9u...| DGA|[29.0,1.0,25.0,0....| DGA|
105oifx8p2skc1av7...| DGA|[31.0,1.0,27.0,0....| DGA|
106vx1l1u97v0953h...| DGA|[30.0,1.0,26.0,0....| DGA|
107dzvs6vwc6bkmkp...| DGA|[30.0,1.0,26.0,0....| DGA|
 107klub.com|Normal|[11.0,1.0,7.0,0.0...| DGA|
10824tf1yor5tqq3s...| DGA|[30.0,1.0,26.0,0....| DGA|
109z94y283zz315cc...| DGA|[30.0,1.0,26.0,0....| DGA|
10bvayh3qigsrij05...| DGA|[30.0,1.0,26.0,0....| DGA|
10c2gqr1baogda1b9...| DGA|[31.0,1.0,27.0,0....| DGA|
10cmato185r2ve145...| DGA|[32.0,1.0,28.0,0....| DGA|
 10dayshairoil.com|Normal|[17.0,1.0,13.0,0....| DGA|
 10ecommerce.com.br|Normal|[18.0,1.0,11.0,0....| DGA|
 10ejemplos.com|Normal|[14.0,1.0,10.0,0....| DGA|
10g5ddmc7r9qj1epu...| DGA|[30.0,1.0,26.0,0....| DGA|
10g88bk41gr5z1tww...| DGA|[30.0,1.0,26.0,0....| DGA|
+--------------------+------+--------------------+--------------+
only showing top 50 rows

Test Error = 0.23358